# Leakage characterization using GST
This tutorial demonstrates how to perform GST on a "leaky-qubit" described by a 3-level (instead of the desired 2-level) system. 

In [1]:
import pygsti
import pygsti.construction as pc
import pygsti.construction.std1Q_XYI as std1Q
import numpy as np
import scipy.linalg as sla
#import pickle

In [2]:
def to_3level_unitary(U_2level):
    U_3level = np.zeros((3,3),complex)
    U_3level[0:2,0:2] = U_2level
    U_3level[2,2] = 1.0
    return U_3level

def unitary_to_gmgate(U):
    return pygsti.tools.change_basis( 
        pygsti.tools.unitary_to_process_mx(U), 'std','gm')

def state_to_gmvec(state):
    pygsti.tools.stdmx_to_gmvec

Us = pygsti.tools.internalgates.get_standard_gatename_unitaries()

In [3]:
mdl_2level_ideal = std1Q.target_model()

In [4]:
rho0 = np.array( [[1,0,0],
                  [0,0,0],
                  [0,0,0]], complex)
E0 = rho0
E1 = np.array( [[0,0,0],
                [0,1,0],
                [0,0,1]], complex)

sslbls = pygsti.obj.StateSpaceLabels(['Qubit+Leakage'],[3])
mdl_3level_ideal = pygsti.obj.ExplicitOpModel(sslbls, 'gm')
mdl_3level_ideal['rho0'] = pygsti.tools.stdmx_to_gmvec(rho0)
mdl_3level_ideal['Mdefault'] = pygsti.obj.TPPOVM([('0',pygsti.tools.stdmx_to_gmvec(E0)),
                                                    ('1',pygsti.tools.stdmx_to_gmvec(E1))])

mdl_3level_ideal['Gi'] = unitary_to_gmgate( to_3level_unitary(Us['Gi']))
mdl_3level_ideal['Gx'] = unitary_to_gmgate( to_3level_unitary(Us['Gxpi2']))
mdl_3level_ideal['Gy'] = unitary_to_gmgate( to_3level_unitary(Us['Gypi2']))

In [5]:
sigmaX = np.array([[0,1],[1,0]],complex)
rot = sla.expm(1j * 0.1 * sigmaX)
Uleakage = np.identity(3,complex)
Uleakage[1:3,1:3] = rot
leakageOp = unitary_to_gmgate(Uleakage)
#print(Uleakage)

#Guess of a model w/just unitary leakage
mdl_3level_guess = mdl_3level_ideal.copy()
mdl_3level_guess['Gi'] = np.dot(leakageOp, mdl_3level_guess['Gi'])
#mdl_3level_guess['Gx'] = np.dot(leakageOp, mdl_3level_guess['Gx'])
#mdl_3level_guess['Gy'] = np.dot(leakageOp, mdl_3level_guess['Gy'])

#Actual model used for data generation (some depolarization too)
mdl_3level_noisy = mdl_3level_ideal.depolarize(op_noise=0.005, spam_noise=0.01)
mdl_3level_noisy['Gi'] = np.dot(leakageOp, mdl_3level_noisy['Gi'])
#mdl_3level_noisy['Gx'] = np.dot(leakageOp, mdl_3level_noisy['Gx'])
#mdl_3level_noisy['Gy'] = np.dot(leakageOp, mdl_3level_noisy['Gy'])

In [6]:
#print(mdl_3level_guess)

In [7]:
# get sequences using expected model
generate_fiducials = False

if generate_fiducials:
    prepfids, measfids = pygsti.algorithms.generate_fiducials(
        mdl_3level_guess, omitIdentity=False, maxFidLength=4, verbosity=4)
    pygsti.io.write_circuit_list("example_files/leakage_prepfids.txt", prepfids)
    pygsti.io.write_circuit_list("example_files/leakage_measfids.txt", measfids)

In [8]:
prepfids = pygsti.io.load_circuit_list("example_files/leakage_prepfids.txt")
measfids = pygsti.io.load_circuit_list("example_files/leakage_measfids.txt")
germs = std1Q.germs
maxLengths = [1,]
expList = pc.make_lsgst_experiment_list(mdl_3level_noisy, prepfids, measfids, germs, maxLengths)
ds = pc.generate_fake_data(mdl_3level_noisy, expList, 1000, 'binomial', seed=1234)

In [9]:
results_2level = pygsti.do_stdpractice_gst(ds, mdl_2level_ideal, prepfids, measfids,
                                           germs, maxLengths, modes="CPTP", verbosity=3)
results_3level = pygsti.do_stdpractice_gst(ds, mdl_3level_ideal, prepfids, measfids,
                                           germs, maxLengths, modes="CPTP,True",
                                           modelsToTest={'True': mdl_3level_noisy}, 
                                           verbosity=4, advancedOptions={'all': {'tolerance': 1e-2}})

-- Std Practice:  Iter 1 of 1  (CPTP) --: 
  --- Circuit Creation ---
     305 sequences created
     Dataset has 305 entries: 305 utilized, 0 requested sequences were missing
  --- Iterative MLGST: Iter 1 of 1  305 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 794.557 (305 data params - 31 model params = expected mean of 274; p-value = 0)
    Completed in 5.4s
    2*Delta(log(L)) = 804.647
    Iteration 1 took 5.4s
    
    Switching to ML objective (last iteration)
    --- MLGST ---
      Maximum log(L) = 395.179 below upper bound of -458095
        2*Delta(log(L)) = 790.358 (305 data params - 31 model params = expected mean of 274; p-value = 0)
      Completed in 1.1s
    2*Delta(log(L)) = 790.358
    Final MLGST took 1.1s
    
  Iterative MLGST Total Time: 6.5s
  --- Re-optimizing logl after robust data scaling ---
    --- MLGST ---
      Maximum log(L) = 395.179 below upper bound of -458095
        2*Delta(log(L)) = 790.357 (305 data params - 31 model 


/Users/enielse/research/pyGSTi/packages/pygsti/objects/estimate.py:531: UserWarning:

Max-model params (305) <= model params (360)!  Using k == 1.



    --- MLGST ---
      --- Outer Iter 0: norm_f = 136.525, mu=0, |J|=1697.85
      Least squares message = Both actual and predicted relative reductions in the sum of squares are at most 0.01
      Maximum log(L) = 136.525 below upper bound of -458095
        2*Delta(log(L)) = 273.05 (305 data params - 161 model params = expected mean of 144; p-value = 4.90231e-10)
      Completed in 1.2s
  -- Performing 'single' gauge optimization on CPTP estimate --
      -- Adding Gauge Optimized (single) --
  -- Conveying 'single' gauge optimization to CPTP.Robust+ estimate --
      -- Adding Gauge Optimized (single) --
-- Std Practice:  Iter 2 of 2  (True) --: 
  --- Circuit Creation ---
     305 sequences created
     Dataset has 305 entries: 305 utilized, 0 requested sequences were missing
  -- Performing 'single' gauge optimization on True estimate --
      -- Adding Gauge Optimized (single) --


In [10]:
pygsti.report.create_standard_report({'two-level': results_2level, 'three-level': results_3level},
                                     "example_files/leakage_report", "Leakage Example Report")

*** Creating workspace ***
*** Generating switchboard ***


/Users/enielse/research/pyGSTi/packages/pygsti/report/factory.py:785: UserWarning:

Idle tomography failed:
Label{layers}



Found standard clifford compilation from std1Q_XYI
*** Generating tables ***
*** Generating plots ***
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
*** Merging into template file ***
Output written to example_files/leakage_report directory
*** Report Generation Complete!  Total time 32.8807s ***


In [11]:
#try a different basis:
gm_basis = pygsti.obj.Basis('gm',3)
   
leakage_basis_mxs = [ np.sqrt(2)/3*(np.sqrt(3)*gm_basis[0] + 0.5*np.sqrt(6)*gm_basis[8]),
                      gm_basis[1], gm_basis[4], gm_basis[7],
                     gm_basis[2], gm_basis[3], gm_basis[5], gm_basis[6],
                     1/3*(np.sqrt(3)*gm_basis[0] - np.sqrt(6)*gm_basis[8]) ]
#for mx in leakage_basis_mxs:
#    pygsti.tools.print_mx(mx)

check = np.zeros( (9,9), complex)
for i,m1 in enumerate(leakage_basis_mxs):
    for j,m2 in enumerate(leakage_basis_mxs):
        check[i,j] = np.trace(np.dot(m1,m2))
assert(np.allclose(check, np.identity(9,complex)))

leakage_basis = pygsti.obj.Basis(name="LeakageBasis", matrices=leakage_basis_mxs, 
                                longname="2+1 level leakage basis", real=True,
                                labels=['I','X','Y','Z','LX0','LX1','LY0','LY1','L'])

def changebasis_3level_model(mdl):
    new_mdl = mdl.copy()
    new_mdl.preps['rho0'] = pygsti.obj.FullSPAMVec(
        pygsti.tools.change_basis(mdl.preps['rho0'].todense(), gm_basis, leakage_basis))
    new_mdl.povms['Mdefault'] = pygsti.obj.UnconstrainedPOVM(
        [('0', pygsti.tools.change_basis(mdl.povms['Mdefault']['0'].todense(), gm_basis, leakage_basis)),
         ('1', pygsti.tools.change_basis(mdl.povms['Mdefault']['1'].todense(), gm_basis, leakage_basis))])
    
    for lbl,op in mdl.operations.items():
        new_mdl.operations[lbl] = pygsti.obj.FullDenseOp(
            pygsti.tools.change_basis(op.todense(), gm_basis, leakage_basis))
    new_mdl.basis = leakage_basis
    return new_mdl

def changebasis_3level_results(results):
    new_results = results.copy()
    for estlbl,est in results.estimates.items():
        for mlbl,mdl in est.models.items():
            if isinstance(mdl,(list,tuple)): #assume a list/tuple of models
                new_results.estimates[estlbl].models[mlbl] = \
                    [ changebasis_3level_model(m) for m in mdl ]
            else:
                new_results.estimates[estlbl].models[mlbl] = changebasis_3level_model(mdl)
    return new_results
    

In [12]:
results_3level_leakage_basis = changebasis_3level_results( results_3level )    

In [13]:
pygsti.report.create_standard_report({'two-level': results_2level, 'three-level': results_3level_leakage_basis},
                                     "example_files/leakage_report", "Leakage Example Report")
                                     #advancedOptions={'autosize': 'none'})

*** Creating workspace ***
*** Generating switchboard ***
Found standard clifford compilation from std1Q_XYI


/Users/enielse/research/pyGSTi/packages/pygsti/report/factory.py:785: UserWarning:

Idle tomography failed:
Label{layers}



*** Generating tables ***
*** Generating plots ***
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
*** Merging into template file ***
Output written to example_files/leakage_report directory
*** Report Generation Complete!  Total time 30.2682s ***


Open the report [here](example_files/leakage_report/main.html)

In [14]:
# use "kite" density-matrix structure
def to_2plus1_superop(superop_2level):
    ret = np.zeros((5,5),'d')
    ret[0:4,0:4] = superop_2level
    ret[4,4] = 1.0 #leave leakage population where it is
    return ret

#Tack on a single extra "0" for the 5-th dimension corresponding
# to the classical leakage level population.
rho0 = np.concatenate( (mdl_2level_ideal.preps['rho0'],[[0]]), axis=0)
E0 = np.concatenate( (mdl_2level_ideal.povms['Mdefault']['0'],[[0]]), axis=0)
E1 = np.concatenate( (mdl_2level_ideal.povms['Mdefault']['1'],[[0]]), axis=0)


sslbls = pygsti.obj.StateSpaceLabels([('Qubit',),('Leakage',)],[(2,),(1,)])
mdl_2plus1_ideal = pygsti.obj.ExplicitOpModel(sslbls, 'gm')
mdl_2plus1_ideal['rho0'] = rho0
mdl_2plus1_ideal['Mdefault'] = pygsti.obj.UnconstrainedPOVM([('0',E0),('1',E1)])

mdl_2plus1_ideal['Gi'] = to_2plus1_superop(mdl_2level_ideal['Gi'])
mdl_2plus1_ideal['Gx'] = to_2plus1_superop(mdl_2level_ideal['Gi'])
mdl_2plus1_ideal['Gy'] = to_2plus1_superop(mdl_2level_ideal['Gi'])

In [15]:
results_2plus1 = pygsti.do_long_sequence_gst(ds, mdl_2plus1_ideal, prepfids, measfids,
                                             germs, maxLengths, verbosity=3,
                                             advancedOptions={"starting point": "target"})

--- Circuit Creation ---
   305 sequences created
   Dataset has 305 entries: 305 utilized, 0 requested sequences were missing
--- Iterative MLGST: Iter 1 of 1  305 operation sequences ---: 
  --- Minimum Chi^2 GST ---
    Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
     groups of ~1 procs each, to distribute over 90 params (taken as 1 param groups of ~90 params).
    --- Outer Iter 0: norm_f = 1.24364e+09, mu=0, |J|=255630
    --- Outer Iter 1: norm_f = 118724, mu=2.46191e+06, |J|=4148.05
    --- Outer Iter 2: norm_f = 84982, mu=820638, |J|=3690.92
    --- Outer Iter 3: norm_f = 77529.7, mu=273546, |J|=3614.72
    --- Outer Iter 4: norm_f = 76388.2, mu=91182, |J|=3594.1
    --- Outer Iter 5: norm_f = 74758.6, mu=30394, |J|=3564.1
    --- Outer Iter 6: norm_f = 73202.6, mu=15306.2, |J|=3596.98
    --- Outer Iter 7: norm_f = 72412.3, mu=5102.08, |J|=3791.87
    --- Outer Iter 8: norm_f = 71574.8, mu=1700.69, |J|=4422.36
    --- Outer Iter 9:

/Users/enielse/research/pyGSTi/packages/pygsti/algorithms/gaugeopt.py:240: UserWarning:

No gauge group specified, so no gauge optimization performed.



    --- Outer Iter 0: norm_f = 171.465, mu=0, |J|=7472.82
    Least squares message = Both actual and predicted relative reductions in the sum of squares are at most 1e-06
    Maximum log(L) = 171.465 below upper bound of -458095
      2*Delta(log(L)) = 342.929 (305 data params - 65 model params = expected mean of 240; p-value = 1.41955e-05)
    Completed in 0.1s
  -- Adding Gauge Optimized (go0) --


In [16]:
pygsti.report.create_standard_report({'two-level': results_2level, 'three-level': results_3level_leakage_basis,
                                      'two+one level': results_2plus1},
                                     "example_files/leakage_report", "Leakage Example Report",
                                     advancedOptions={'autosize': 'none'})

*** Creating workspace ***
*** Generating switchboard ***
Found standard clifford compilation from std1Q_XYI


/Users/enielse/research/pyGSTi/packages/pygsti/report/factory.py:785: UserWarning:

Idle tomography failed:
Label{layers}



*** Generating tables ***


/Users/enielse/research/pyGSTi/packages/pygsti/extras/rb/theory.py:200: UserWarning:

Output may be unreliable because the model is not approximately trace-preserving.



*** Generating plots ***
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
*** Merging into template file ***
Output written to example_files/leakage_

Open the report [here](example_files/leakage_report/main.html)